In [39]:
# the URL of the spreadsheet, for posting
post_url = 'https://ethercalc.net/_/MyWeirdName23'

# set a reminder of when the last data was stored
last_time = datetime.now() - timedelta(hours=2)

# check if it is daytime
def is_daylight(): 
    if (datetime.now().hour > 8) and (datetime.now().hour < 20):
        return True
    return False

# a widget to monitor the data being posted
monitor = widgets.Text(
    value="No recent post yet",
    placeholder="blank",
    description="Latest data:",
    disabled = False
    )

# here we grab the data results from comm, check if it is time to post, then post. 
def post_stuff(result):
    global last_time
    if is_daylight():
        wait_time = 14*60 # in daytime, we post every 15 minute (wait 14). 
    else:
        wait_time = 59*60 # in nighttime, post every 60 minutes (wait 59)
    if (datetime.now()-last_time).seconds < wait_time:
        return
    last_time = datetime.now()
    data = datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
    data += ',' + str(result['temp'])
    data += ',' + str(result['humidity'])
    data += ',' + str(result['ratio'])
    data += ',' + str(result['lux'])
    monitor.value = data    
    r = requests.post(post_url, data= data) ## we could return a status code here...

## Step 5: Open a comms channel 

This connects the Javascript and Python engines.

Javascript (front end) and Python (back end) run separately on the Jupyter system. We open a Comms channel between the two so JS can pass the sensor data to Python, then Python posts it onto the web.  

There is an issue that Python runs as a single thread in Jupyter. So if you put a "sleep" command in a Python loop, no other code can access the Jupyter kernel during that loop. The sensor data is then not recorded. 

The solution is to create a timer loop in Javascript, so that every 5 minutes it sends a message to the Python kernel. Basically, the message says "hey Python, here is the latest data. Please save it to the cloud." Then the Python code can run and save the data. 

This way there is no loop or sleep in Python, so nothing to interupt. The loop is in JS (See "setIntevalTimer in the JS code below).
